In [32]:
!pip install transformers huggingface_hub
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes
!pip install accelerate
!pip install datasets

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-vqn31pz2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-vqn31pz2
  Resolved https://github.com/huggingface/transformers.git to commit 3476c19e91a882fab8c76970a8798890a3f0f299
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for huggingface-hub<1.0,>=0.23.2 from https://files.pythonhosted.org/packages/d5/ce/1f8e61cd63175cc2e79233b954b1c4e85363c788fb3a1fa23c87a25c9b81/huggingface_hub-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.20,>=0.19 from https://files.pythonhosted.org/packages/a7/03/fb50fc03f86016b227a967c8d474f90230c885c0d18f78acdfda7a96ce56/tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43

In [33]:
from langchain_core.language_models.llms import LLM
import transformers
import torch
import pandas as pd
from pydantic import Field

In [34]:
class CustomLLM(LLM):

	# model_id: str
	# pipeline: Any

	model_id: str

	def __init__(self, model_id:str):
		# print("Loading model with id: ", model_id)
		super().__init__(model_id=model_id)
		self.model_id = model_id
		self.pipeline = transformers.pipeline(
			"text-generation",
			model=self.model_id,
			device_map="auto",
			model_kwargs={
			"torch_dtype": torch.bfloat16,
			"quantization_config": {"load_in_4bit": True},
			"low_cpu_mem_usage": True,
			}
		)
		
		
	
	def _call(self, prompt):

		messages = [
			{"role": "user", "content": prompt},
		]

		outputs = self.pipeline(
			messages,
			max_new_tokens=512
		)
		return outputs[0]["generated_text"][-1]["content"]
	
	@property
	def _llm_type(self) -> str:
		"""Get the type of language model used by this chat model. Used for logging purposes only."""
		return self.model_id


In [35]:
llm = CustomLLM(model_id="microsoft/Phi-3.5-mini-instruct")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`